# Spectral Clustering

In [2]:
from itertools import permutations
import numpy as np
import scipy as sp
import scipy.sparse.linalg
import scipy.cluster.vq

In [3]:
A = np.loadtxt('../data/processed/usps.csv', delimiter=',')

In [16]:
inds = A[:, -1] < 6
X = A[inds, :-2]
Y = A[inds, -1].astype(int)

In [17]:
k = len(np.unique(Y))
n, d = X.shape
n, d

(4240, 254)

In [18]:
W = np.empty((n, n))
for i in range(n):
    for j in range(i,n):
#         val = np.e ** (-1 * np.linalg.norm(X[i] - X[j]) ** 2)
        val = np.linalg.norm(X[i] - X[j]) ** 2
        W[i, j] = val
        W[j, i] = val

In [19]:
ww = W.sum(axis=0)
D = np.diag(ww)
D_ = np.diag(1 / np.sqrt(ww))
L = np.identity(n) - D_.dot(W).dot(D_)

In [20]:
V, Z = sp.linalg.eigh(L, eigvals=(n-2, n-1))

In [21]:
Z_ = sp.cluster.vq.whiten(Z)
centroids, distortion = sp.cluster.vq.kmeans(Z_, k)

In [22]:
centroids, distortion

(array([[-0.38260635,  0.87423866],
        [ 0.68671565, -1.41447699],
        [-1.29956181, -0.11793806],
        [ 0.16349659, -0.22927704],
        [ 1.30778268,  1.31199514]]), 0.44527827954764854)

In [23]:
y_hat = np.zeros(n, dtype=int)
for i in range(n):
    dists = np.array([np.linalg.norm(Z_[i] - centroids[c]) for c in range(k)])
    y_hat[i] = np.argmin(dists)

In [24]:
perms = []
for p in permutations(np.arange(1, k+1)):
    P = dict()
    for i in range(k):
        P[i] = p[i]
    perms.append(P)

In [25]:
accuracy = np.zeros(len(perms))
for i in range(len(perms)):
    yy = y_hat.copy()
    for k, v in perms[i].items():
        yy[y_hat == k] = v
    accuracy[i] = (Y == yy).sum() / n * 100
accuracy.max()

62.405660377358487